In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.iloc[1000:1020]

ArrowInvalid: Error creating dataset. Could not read schema from '../data/processed/rides_2023_01.parquet'. Is this a 'parquet' file?: Could not open Parquet input source '../data/processed/rides_2023_01.parquet': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
rides.info()

In [ ]:
rides["pickup_hour"] = rides["pickup_datetime"].dt.floor('h')
rides.tail()

In [ ]:
agg_rides = rides.groupby(["pickup_hour", "pickup_location_id"]).size().reset_index()
agg_rides.rename(columns={0: "rides"}, inplace=True)
agg_rides.tail()

In [ ]:
# Original dataset
data = {
    "hour": ["2025-01-28 08:00", "2025-01-28 08:00", "2025-01-28 09:00", "2025-01-28 10:00"],
    "location_id": [1, 2, 1, 2],
    "rides": [10, 5, 8, 12]
}
df = pd.DataFrame(data)
df["hour"] = pd.to_datetime(df["hour"])

df

In [ ]:
# get all the hours
hour_col = "hour"
full_hours = pd.date_range(
    start=df[hour_col].min(),
    end=df[hour_col].max(),
    freq="h"
)

full_hours

In [ ]:
# unique location ids
location_col = "location_id"
all_locations = df[location_col].unique()
all_locations

In [ ]:
# Create a list to store the complete data
complete_data = []
rides_col = "rides"
# Iterate over all combinations of hours and locations
for hour in full_hours:
    for location in all_locations:
        # Check if the combination exists in the original DataFrame
        subset = df[(df[hour_col] == hour) & (df[location_col] == location)]
        if not subset.empty:
            # If the combination exists, append the row
            complete_data.append(subset.iloc[0].to_dict())
        else:
            # If the combination is missing, append a row with 0 rides
            complete_data.append({hour_col: hour, location_col: location, rides_col: 0})

df_complete = pd.DataFrame(complete_data)
df_complete

In [ ]:
df_complete.sort_values(["location_id", "hour"])

In [ ]:
# Create full combinations of hours and locations
full_combinations = pd.DataFrame(
    [(hour, location) for hour in full_hours for location in all_locations],
    columns=["hour", "location_id"]
)

full_combinations

In [ ]:
df

In [ ]:
# Merge with the original dataset
merged_df = pd.merge(full_combinations, df, on=["hour", "location_id"], how="left")
merged_df

In [ ]:
merged_df["rides"] = merged_df["rides"].fillna(0).astype(int)
merged_df

In [ ]:
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    # Ensure the hour column is in datetime format
    df[hour_col] = pd.to_datetime(df[hour_col])

    # Get the full range of hours (from min to max) with hourly frequency
    full_hours = pd.date_range(
        start=df[hour_col].min(),
        end=df[hour_col].max(),
        freq="h"
    )

    # Get all unique location IDs
    all_locations = df[location_col].unique()

    # Create a DataFrame with all combinations of hours and locations
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )

    # Merge the original DataFrame with the full combinations DataFrame
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')

    # Fill missing rides with 0
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)

    return merged_df

In [ ]:
agg_rides

In [ ]:
hour_col = "pickup_hour"
location_col = "pickup_location_id"
rides_col = "rides"
agg_data_filled = fill_missing_rides_full_range(agg_rides, hour_col, location_col, rides_col).sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

In [ ]:
pd.set_option('display.max_rows', 2000)
agg_data_filled.head(1489)


In [ ]:
pip install plotly

In [ ]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="pickup_location_id",
        template="none"
    )

    fig.show()

In [ ]:
plot_rides(agg_data_filled, locations=[42, 43])

In [ ]:
month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"

agg_data_filled.to_parquet(path)